# Extracción y Procesamiento de Información sobre Artículos científicos en ElSevier

Programa de extracción de artículos científicos publicados en ElSevier a través de la API oficial publicada por esta editorial

## Instalación de librerías para el consumo y procesamiento de la información

1. ELSevier API: Se usa el API publicado en https://github.com/ElsevierDev/elsapy que permite la extracción de la información de los artículos publicados

In [1]:
!pip install elsapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2. Pandas: Permite la manupulación de conjuntos de datos en memoria para un procesamiento más rápido y versátil



In [2]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


3. Beautiful Soap: Librería usada para tareas de procesamiento de contenido web o Web Scrapping de información semi-estructurada en contenido con formato html/xml  

In [ ]:
!pip install beautifulsoup4

4. URLLib versión 3: Permite establecer conexión y extraer el contenido de sitios web 

In [ ]:
!pip install urllib3

5. LXML: Permite manipular contenido en formato XML de forma fácil

In [ ]:
!pip install lxml

6. HTML5Lib: Librería que permite utilizar las propiedades de objetos HTML5 a fin de poder estructurar información en páginas web con este tipo de codificación

In [ ]:
!pip install html5lib

## Programa de extracción

### Importación de las librerías a ser usadas

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
import pandas as pd
import time
import copy
import requests
from bs4 import BeautifulSoup

### Carga del archivo de configuración de seguridad para el llamado del API ElSevier

In [ ]:
## Carga de archivo de configuración en formato json
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

In [ ]:
## Inicialización del objeto de conexión usando el valor de la API Key la cual se obtiene de registrándote en el site de desarrolladores: https://dev.elsevier.com/apikey/manage
client = ElsClient(config['apikey'])

### Definición de Función de Extracción Detallada de Información por Artículo Científico
Extrae la información individual de cada artículo científico a partir del site  publicados en el indexador Scopus

In [ ]:
def get_scopus_info(id, URL, API_KEY):
    resp = requests.get(URL,headers={'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                                     'X-ELS-APIKey': config['apikey'],
                                     'Cookie': '',
                                     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0'})
    soup = BeautifulSoup(resp.text, 'lxml')
    result = dict({"id": id})
    if soup.find(id='abstractSection'):
        result['abstract'] = soup.find(id='abstractSection').p.text
    if soup.find(id='authorKeywords'):
        keys = ''
        for x in soup.find(id='authorKeywords').find_all('span'):
            if keys != '':
                keys += ','+x.text
            else:
                keys = x.text
        result['keywords'] = keys
    
    return result

### Extracción de Información - Salud
Se realiza un procesamiento de información sobre los artículos científicos relacionados al uso de Inteligencia Artificial en Políticas de Salud en América Latina.
Para ello se usa el motor de búsqueda Scopus cuyos parámetros de búsqueda se encuentran documentados en https://dev.elsevier.com/sc_search_tips.html

In [ ]:
## Búsqueda de artículos que cumplan con la mención de las siguientes palabras clave dentro del "Título, Abstract y Términos Clave":
### ("policy" OR "strategy")
### AND
### ("artificial intelligence" OR "machine learning" OR "NLP" OR "computer vision")
### AND
### ("latin america" OR "latinamerica" OR "Venezuela" OR "Colombia" OR "Peru" OR "Brazil" OR "Chile" OR "Argentina" OR "Uruguay" OR "Paraguay")
### AND
### Áreas de: Salud(HEAL) OR Psicología(PSYC) OR Medicina(MEDI) OR Neurología(NEUR) OR Enfermería(NURS) OR Farmacia(PHAR) 
### Publicaciones a partir del 2011
doc_srch = ElsSearch("(TITLE-ABS-KEY(policy) OR TITLE-ABS-KEY(strategy)) AND (TITLE-ABS-KEY(artificial intelligence) OR TITLE-ABS-KEY(machine learning) OR TITLE-ABS-KEY(NLP) OR TITLE-ABS-KEY(computer vision)) AND (TITLE-ABS-KEY(latin america) OR TITLE-ABS-KEY(latinamerica) OR TITLE-ABS-KEY(Venezuela) OR TITLE-ABS-KEY(Colombia) OR TITLE-ABS-KEY(Ecuador) OR TITLE-ABS-KEY(Peru) OR TITLE-ABS-KEY(Brazil) OR TITLE-ABS-KEY(Chile) OR TITLE-ABS-KEY(Argentina) OR TITLE-ABS-KEY(Uruguay) OR TITLE-ABS-KEY(Paraguay)) AND (SUBJAREA(HEAL) OR SUBJAREA(PSYC) OR SUBJAREA(MEDI) OR SUBJAREA(NEUR) OR SUBJAREA(NURS) OR SUBJAREA(PHAR)) AND PUBYEAR > 2010",'scopus')
doc_srch.execute(client, get_all = True)
print ("doc_srch has", len(doc_srch.results), "results.")

doc_srch has 56 results.


In [ ]:
### Se almacena el resultado en formato CSV con los metadatos básicos de los artículos obtenidos (Ejecutado en el 8 de noviembre del 2021)
### El detalle de los campos almacenados se encuentran en: https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl
doc_srch.results_df.to_csv('./data/elsevier_scopus_09.csv')

In [ ]:
### Por cada uno de los artículos obteneidos se enriquece la información a partir de la función desarrollada anteriormente
df = pd.read_csv('./data/elsevier_scopus_09.csv')
results_info_list = list()
for i, row in df.iterrows():
    results_info = get_scopus_info(row['dc:identifier'], json.loads(row['link'].replace("\'", "\""))['scopus'], config['apikey'])
    if results_info:
        results_info_list.append(copy.deepcopy(results_info))

df = pd.DataFrame(results_info_list)
df.to_csv('./data/elsevier_scopus_09_detail_01.csv')

### Extracción de Información - Educación
Se hace uso semejante pero enfocando el área de Educación

In [ ]:
## Búsqueda de artículos que cumplan con la mención de las siguientes palabras clave dentro del "Título, Abstract y Términos Clave":
### ("policy" OR "strategy")
### AND
### ("artificial intelligence" OR "machine learning" OR "NLP" OR "computer vision")
### AND
### ("latin america" OR "latinamerica" OR "Venezuela" OR "Colombia" OR "Peru" OR "Brazil" OR "Chile" OR "Argentina" OR "Uruguay" OR "Paraguay")
### AND
### ("education" OR "edu" OR "school") -> ello debido a que en el API no se pudo identificar el uso del área de educación en específico
### Publicaciones a partir del 2011
doc_srch = ElsSearch("(TITLE-ABS-KEY(policy) OR TITLE-ABS-KEY(strategy)) AND (TITLE-ABS-KEY(artificial intelligence) OR TITLE-ABS-KEY(machine learning) OR TITLE-ABS-KEY(NLP) OR TITLE-ABS-KEY(computer vision)) AND (TITLE-ABS-KEY(latin america) OR TITLE-ABS-KEY(latinamerica) OR TITLE-ABS-KEY(Venezuela) OR TITLE-ABS-KEY(Colombia) OR TITLE-ABS-KEY(Ecuador) OR TITLE-ABS-KEY(Peru) OR TITLE-ABS-KEY(Brazil) OR TITLE-ABS-KEY(Chile) OR TITLE-ABS-KEY(Argentina) OR TITLE-ABS-KEY(Uruguay) OR TITLE-ABS-KEY(Paraguay)) AND (TITLE-ABS-KEY(education) OR TITLE-ABS-KEY(edu) OR TITLE-ABS-KEY(school)) AND PUBYEAR > 2010",'scopus')
doc_srch.execute(client, get_all = True)
print ("doc_srch has", len(doc_srch.results), "results.")

doc_srch has 47 results.


In [ ]:
### Se almacena el resultado en formato CSV con los metadatos básicos de los artículos obtenidos (Ejecutado en el 8 de noviembre del 2021)
### El detalle de los campos almacenados se encuentran en: https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl
doc_srch.results_df.to_csv('./data/elsevier_scopus_10.csv')

In [ ]:
### Por cada uno de los artículos obteneidos se enriquece la información a partir de la función desarrollada anteriormente
df = pd.read_csv('./data/elsevier_scopus_10.csv')
results_info_list = list()
for i, row in df.iterrows():
    results_info = get_scopus_info(row['dc:identifier'], json.loads(row['link'].replace("\'", "\""))['scopus'], config['apikey'])
    if results_info:
        results_info_list.append(copy.deepcopy(results_info))

df = pd.DataFrame(results_info_list)
df.to_csv('./data/elsevier_scopus_10_detail_01.csv')

### Extracción de Información - Agricultura
Se hace uso semejante pero enfocando el área de Agricultura

In [ ]:
## Búsqueda de artículos que cumplan con la mención de las siguientes palabras clave dentro del "Título, Abstract y Términos Clave":
### ("policy" OR "strategy")
### AND
### ("artificial intelligence" OR "machine learning" OR "NLP" OR "computer vision")
### AND
### ("latin america" OR "latinamerica" OR "Venezuela" OR "Colombia" OR "Peru" OR "Brazil" OR "Chile" OR "Argentina" OR "Uruguay" OR "Paraguay")
### AND
### Áreas de: Agricultura(AGRI) 
### Publicaciones a partir del 2011
doc_srch = ElsSearch("(TITLE-ABS-KEY(policy) OR TITLE-ABS-KEY(strategy)) AND (TITLE-ABS-KEY(artificial intelligence) OR TITLE-ABS-KEY(machine learning) OR TITLE-ABS-KEY(NLP) OR TITLE-ABS-KEY(computer vision)) AND (TITLE-ABS-KEY(latin america) OR TITLE-ABS-KEY(latinamerica) OR TITLE-ABS-KEY(Venezuela) OR TITLE-ABS-KEY(Colombia) OR TITLE-ABS-KEY(Ecuador) OR TITLE-ABS-KEY(Peru) OR TITLE-ABS-KEY(Brazil) OR TITLE-ABS-KEY(Chile) OR TITLE-ABS-KEY(Argentina) OR TITLE-ABS-KEY(Uruguay) OR TITLE-ABS-KEY(Paraguay)) AND (SUBJAREA(AGRI)) AND PUBYEAR > 2010",'scopus')
doc_srch.execute(client, get_all = True)
print ("doc_srch has", len(doc_srch.results), "results.")

doc_srch has 39 results.


In [ ]:
### Se almacena el resultado en formato CSV con los metadatos básicos de los artículos obtenidos (Ejecutado en el 8 de noviembre del 2021)
### El detalle de los campos almacenados se encuentran en: https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl
doc_srch.results_df.to_csv('./data/elsevier_scopus_11.csv')

In [ ]:
### Por cada uno de los artículos obteneidos se enriquece la información a partir de la función desarrollada anteriormente
df = pd.read_csv('./data/elsevier_scopus_11.csv')
results_info_list = list()
for i, row in df.iterrows():
    results_info = get_scopus_info(row['dc:identifier'], json.loads(row['link'].replace("\'", "\""))['scopus'], config['apikey'])
    if results_info:
        results_info_list.append(copy.deepcopy(results_info))

df = pd.DataFrame(results_info_list)
df.to_csv('./data/elsevier_scopus_11_detail_01.csv')

## Estandarización para explotación de Información sobre Salud
Se realiza un proceso de estandarización de la información para que pueda ser ingresada en un motor de base de datos relacional y de esta forma se permita su visualización en Superset

### Importación de librerías para procesamiento

In [ ]:
import pandas as pd
import numpy as np
import json
import ast

### Carga de la información sobre Salud

In [ ]:
df01 = pd.read_csv('./data/elsevier_scopus_09.csv')
df02 = pd.read_csv('./data/elsevier_scopus_09_detail_01.csv')

### Unificación de la metadata original y la información detallada extraída por Web Scrapping

In [ ]:
## Unificación de ambos datasets a partir de su identificador
df01.drop(['Unnamed: 0'], axis=1, inplace=True)
df02.drop(['Unnamed: 0'], axis=1, inplace=True)
df02.rename(columns = {'id':'dc:identifier'}, inplace = True)
df_total = pd.merge(df01,df02, on='dc:identifier')

In [ ]:
## Almacenamiento de la información unificada en formato csv
df_total.to_csv('./data/elsevier_scopus_salud_total.csv', index=False)

### Extracción de la información semi-estructura de afiliaciones

In [ ]:
## Se extrae la información de las afiliaciones por cada uno de los artículos para estructurarlo en una tabla
result_affiliation_list = list()
for i,row in df_total.iterrows():
    if row['affiliation']:
        for affiliation_dict in ast.literal_eval(row['affiliation']):
            result_affiliation_list.append({'dc:identifier':row['dc:identifier'],
                                            'prism:coverDate': row['prism:coverDate'],
                                           'affilname': affiliation_dict['affilname'],
                                           'affiliation-city': affiliation_dict['affiliation-city'],
                                           'affiliation-country': affiliation_dict['affiliation-country']})
df_affiliation = pd.DataFrame(result_affiliation_list)

In [ ]:
## Almacenamiento de la información de afiliaciones en formato csv
df_affiliation.to_csv('./data/elsevier_scopus_salud_affiliation.csv', index=False)

### Se extraen las palabras clave de cada articulo 

In [ ]:
## Se extrae la información de las palabras clave por cada uno de los artículos para estructurarlo en una tabla
result_keywords_list = list()
for i,row in df_total.iterrows():
    if row['keywords'] and isinstance(row['keywords'], str):
        for keyword in row['keywords'].split(','):
            result_keywords_list.append({'dc:identifier':row['dc:identifier'],
                                         'prism:coverDate': row['prism:coverDate'],
                                         'keyword': keyword.strip().lower()})
df_keywords = pd.DataFrame(result_keywords_list)

In [ ]:
## Almacenamiento de la información de las palabras clave en formato csv
df_keywords.to_csv('./data/elsevier_scopus_salud_keywords.csv', index=False)

### Creación del repositorio en SQLLite
Se crea la base de datos la cual se popula utilizando el IDE que ofrece Superset

In [ ]:
!C:\sqlite\sqlite3.exe ./data/csv_database.db